In [30]:
import sys
sys.path.append("..")
sys.path.append("../lib")
sys.path.append("../data")
sys.path.append("../services")
sys.path.append("../embedding_storage")
sys.path.append("../services/evaluation")
import importlib
import preproc as pre
importlib.reload(pre)
import search_service as search
importlib.reload(search)
import os
import pandas as pd
import evaluate_summarization as eval_summa
importlib.reload(eval_summa)
import summarize as summa

# Create dataset

In [2]:
dataset = pre.make_dataset_from_txt('../data/extracted-text/')

In [15]:
dataset['input_redact'] = dataset['input'].apply(pre.redact_specified_parts)

In [16]:
dataset.head()

,type,title,input,data_source,input_redact
0,ABSTRACT,ABSTRACT,ABSTRACT\n Medicine relies on the use of pharm...,Emerging Frontiers in Drug Delivery.txt,ABSTRACT\n Medicine relies on the use of pharm...
1,TITLE_PARAGRAPH,INTRODUCTION,Medicine relies on the use of pharmacologicall...,Emerging Frontiers in Drug Delivery.txt,Medicine relies on the use of pharmacologicall...
2,TITLE_PARAGRAPH,DRUG DELIVERY SYSTEMS FOR CONTROLLED,RELEASE One important class of DDS is controll...,Emerging Frontiers in Drug Delivery.txt,RELEASE One important class of DDS is controll...
3,TITLE_PARAGRAPH,SYSTEMIC RNA DELIVERY,RNAs can manipulate gene expression through se...,Emerging Frontiers in Drug Delivery.txt,RNAs can manipulate gene expression through se...
4,TITLE_PARAGRAPH,DRUG DELIVERY SYSTEMS FOR LOCALIZED THERAPY,One potential limitation to systemic administr...,Emerging Frontiers in Drug Delivery.txt,One potential limitation to systemic administr...


# Extract abstract per paper

In [ ]:
combined_abstracts = []
for article in list(set(dataset.data_source)):
    combined_abstracts.append(' '.join(dataset[(dataset.data_source == article) &\
                                  (dataset.type == 'ABSTRACT')].input_redact.tolist()))     

In [ ]:
len(combined_abstracts)

# Extract paragraphs per paper

In [29]:
paragraphs_all_papers = {}

for article in dataset['data_source'].unique():
    paragraphs = dataset[(dataset['data_source'] == article) & (dataset['type'] != 'ABSTRACT')]['input_redact'].tolist()
    paragraphs = [paragraph for paragraph in paragraphs if len(paragraph) > 100]
    paragraphs = pre.split_and_filter_paragraphs(paragraphs)

    article_key = article if isinstance(article, str) else str(article)
    paragraphs_all_papers[article_key] = paragraphs

In [28]:
paragraphs_all_papers.keys()

dict_keys(['Emerging Frontiers in Drug Delivery.txt', 'Efficacy and Safety of the mRNA-1273 SARS-CoV-2 Vaccine.txt', '82_2020_217.txt', 's41591-022-02061-1.txt', 'pharmaceutics-12-00102-v2.txt', 'Safety and Efficacy of the BNT162b2 mRNA Covid-19 Vaccine.txt', 'PMC8325057.txt', 's41392-022-00950-y.txt', 'PMC8198544.txt', 'mRNA vaccines — a new era.txt', 'nanomaterials-10-00364-v2.txt', 's41392-022-01007-w.txt', 'biomedicines-11-00308-v2.txt'])

# Summary per paper

In [ ]:
summaries = []
for key in paragraphs_all_papers.keys():
    summaries.append(summa.llama_2_7b_q2_default(paragraphs_all_papers[key]))

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../models/llama-2-7b.Q2_K.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q2_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q3_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q3_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q3_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q2_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q3_K     [  4096,  4096,     1,     1 ]
llama

# Evaluate summary

In [ ]:
for summary, abstract in zip(summaries, combined_abstracts):
    eval_summa.evaluate_summarization(summary, abstract)